In [30]:
import re
import sys
import pprint
import json
from IPython.display import display
from colorama import Fore, Back, Style

In [51]:
with open('./res/lab1.json') as lab1_file:
    data = json.load(lab1_file)
chain = data['chain']
service_words = data['tables']['service_words']
operations = data['tables']['operations']
separators = data['tables']['separators']
constants = data['tables']['constants']
identifiers = data['tables']['identifiers']

stack = []
result = []

In [54]:
def get_priority(operation):
    priorities = [
        ['('],
        [')'],
        ['||'],
        ['&&'],
        ['!'],
        ['<', '>', '<=', '>=', '==', '!='],
        ['+', '-'],
        ['*', '/'],
        ['^']
    ]
    i = 0
    while operation not in priorities[i]:
        i+=1
        if i > len(priorities - 1):
            return False
    return i

In [40]:
def get_token_name(token):
    global chain, service_words, operations, separators, constants, identifiers
    codes = ['W', 'O', 'R', 'C', 'I']
    tables = [service_words, operations, separators, constants, identifiers]
    return tables[codes.index(token[0])][token[1]]

In [50]:
for token in chain:
    print(stack, Fore.WHITE + str(get_token_name(token)) + Style.RESET_ALL)
    
    if get_token_name(token) in operations
    
    print(stack)
    print(Back.WHITE, str(result), Style.RESET_ALL)
    print()

[] void
[]
 [] 

[] ,
[]
 [] 

[] {'type': 'identifier', 'name': 'func'}
[]
 [] 

[] )
[]
 [] 

[] float
[]
 [] 

[] ,
[]
 [] 

[] {'type': 'identifier', 'name': 'f1'}
[]
 [] 

[] :
[]
 [] 

[] ,
[]
 [] 

[] int
[]
 [] 

[] ,
[]
 [] 

[] {'type': 'identifier', 'name': 'f2'}
[]
 [] 

[] [
[]
 [] 

[] ,
[]
 [] 

[] {
[]
 [] 

[] ,
[]
 [] 

[] return
[]
 [] 

[] ,
[]
 [] 

[] {'type': 'integer', 'value': '0'}
[]
 [] 

[] (
[]
 [] 

[] int
[]
 [] 

[] ,
[]
 [] 

[] main
[]
 [] 

[] )
[]
 [] 

[] [
[]
 [] 

[] ,
[]
 [] 

[] int
[]
 [] 

[] ,
[]
 [] 

[] {'type': 'identifier', 'name': 'a'}
[]
 [] 

[] :
[]
 [] 

[] ,
[]
 [] 

[] {'type': 'identifier', 'name': 'b'}
[]
 [] 

[] ]
[]
 [] 

[] {'type': 'integer', 'value': '5'}
[]
 [] 

[] '
[]
 [] 

[] :
[]
 [] 

[] ,
[]
 [] 

[] {'type': 'identifier', 'name': 'c'}
[]
 [] 

[] (
[]
 [] 

[] ,
[]
 [] 

[] char
[]
 [] 

[] ,
[]
 [] 

[] {'type': 'identifier', 'name': 'd'}
[]
 [] 

[] ]
[]
 [] 

[] {'type': 'integer', 'value': '1'}
[]
 [] 

[] '
[]